# 6. Optimizing Model Parameters

앞선 노트북을 통해 모델 생성, 데이터 셋 구축, back propagaion 과정을 살펴보았습니다.

따라서 이제 파라미터를 최적화 해 모델을 학습, 검증, 테스트 해 봅시다.

모델 학습은 iteractive 한 process로 

1epoc 마다 모델은 output 에 대한 추측을 하고, 추측의 오류(손식)를 계산하고, 파라미터에 대한 오류의 파생물(derivatives)를 수집하고, 경사하강법(gradient descent)를 활용해 이러한 파라미터들을 최적화 한다.



## 6.1 Prerequisite Code

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [34]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits


In [35]:
model = NeuralNetwork()

## 6.2 Hyperparameters

Hyperparameters 는 모델 optimization을 제어할 수 있는 adjustable(조정 가능)한 파라미터이다.

이 하이퍼파라미터는 모델 학습과 수렴률(convergence rates)에 영향을 줄 수 있다.

아래 3가지 파라미터를 정의 해보자.

* `Number of Epochs` : 데이터 집합에 대해 반복하는 횟수
* `Batch Size` : 파라미터가 업데이트되기 전에 네트워크를 통해 전파되는 **데이터 샘플 수**
* `Learing Rate` : 각 배치/에포트에서 모델 매개변수를 업데이트하는 양입니다. 값이 작으면 학습 속도가 느려지는 반면 값이 크면 훈련 중에 예측할 수 없는 동작이 발생할 수 있다.

In [20]:
learning_rate = 1e-3
batch_size = 64 # 1 batch에 64개 samples
epochs = 5

## 6.3 Optimization Loop

하이퍼 파라미터를 설정하면 최적화 루프를 사용하여 모델을 훈련하고 최적화할 수 있다. 최적화 루프의 각 반복은 epohc 라고 불린다.

한 `epoch` 은 두 개의 주요 부분으로 구성된다

* `Train Loop` : 학습 데이터셋에 대해 반복하고 최적의 매개 변수로 수렴하려고 시도한다.
* `Validation/Test Loop` : 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복(iterate)합니다.


학습 단계에서 사용되는 몇 가지 개념을 간략하게 숙지해 보겠습니다. 



### 6.3.1 loss function

`손실 함수(loss function)`는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. 

주어진 데이터 샘플을 입력으로 계산한 **예측과 정답(label)을 비교**하여 손실(loss)을 계산합니다.

<br>

일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 [`nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) 평균 제곱 오차((MSE; Mean Square Error))나 분류(classification)에 사용하는 [`nn.NLLLoss` ](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood), 그리고 `nn.LogSoftmax`와 `nn.NLLLoss`를 합친 [`nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) 등이 있습니다.

<br>

모델의 출력 logit을 `nn.CrossEntropyLoss`에 전달하여 로짓(logit)을 정규화하고 예측 오류를 계산합니다.


In [21]:
# 손실 함수를 초기화 합니다.
loss_fn = nn.CrossEntropyLoss()

## 6.4 Optimizer

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. ***최적화 알고리즘*** 은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의합니다. 

<br>

**모든 최적화 절차(logic)**는 `optimizer` 객체에 **캡슐화(encapsulated)**됩니다. 여기서는 `SGD` 옵티마이저를 사용하고 있으며, PyTorch에는 `ADAM`이나 `RMSProp`과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있습니다.

<br>

학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화합니다.

In [36]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

⛔️ init 변수 오타 -> model.parameters() 생성 x

<h3> training loop 에서 최적화는 세단계로 이뤄집니다


>1. `optimizer.zero_grad()`를 호출하여 **모델 parameters의 변화도를 재설정**합니다. 기본적으로 **변화도는 더해지기(add up)** 때문에 중복 계산을 막기 위해 **반복할 때마다 명시적으로 0으로 설정**합니다.

>2. `loss.backwards()`를 호출하여 **예측 손실(prediction loss)을 역전파**합니다. PyTorch는 ***각 매개변수에 대한 손실의 변화도를 저장***합니다.

>3. 변화도를 계산한 뒤에는 `optimizer.step()`을 호출하여 **역전파 단계에서 수집된 변화도로 매개변수를 조정**합니다.

## 6.5 Full Implementation
We define train_loop that loops over our optimization code, and test_loop that evaluates the model’s performance against our test data.

In [37]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]") # display loss & the degree of progress


def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X) # prediction value
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to train_loop and test_loop. Feel free to increase the number of epochs to track the model’s improving performance.

In [38]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
------------------------------
loss : 2.318551 [    0/60000]
loss : 2.300772 [ 6400/60000]
loss : 2.280788 [12800/60000]
loss : 2.263073 [19200/60000]
loss : 2.252583 [25600/60000]
loss : 2.226530 [32000/60000]
loss : 2.229961 [38400/60000]
loss : 2.206465 [44800/60000]
loss : 2.211864 [51200/60000]
loss : 2.150726 [57600/60000]
Test Error: 
 Accuracy: 39.4%, Avg loss: 2.161573 

Epoch 2
------------------------------
loss : 2.189877 [    0/60000]
loss : 2.174411 [ 6400/60000]
loss : 2.119974 [12800/60000]
loss : 2.125489 [19200/60000]
loss : 2.080481 [25600/60000]
loss : 2.023562 [32000/60000]
loss : 2.054281 [38400/60000]
loss : 1.980469 [44800/60000]
loss : 2.005359 [51200/60000]
loss : 1.903097 [57600/60000]
Test Error: 
 Accuracy: 50.9%, Avg loss: 1.912901 

Epoch 3
------------------------------
loss : 1.962109 [    0/60000]
loss : 1.928234 [ 6400/60000]
loss : 1.817432 [12800/60000]
loss : 1.847195 [19200/60000]
loss : 1.737925 [25600/60000]
loss : 1.684534 [32000/60000]

### 외) Correct 구하는 과정

In [48]:
for it, (x, y) in enumerate(test_dataloader):
  X = x
  Y = y
  if it == 0:
    break

with torch.no_grad():
  pred = model(X) # prediction value
  test_loss = loss_fn(pred, y).item()
  correct = (pred.argmax(1) == y).type(torch.float).sum().item()

print(correct)

42.0


In [49]:
pred.argmax(1) == y

tensor([ True,  True,  True,  True,  True,  True,  True, False, False,  True,
        False,  True,  True,  True,  True,  True,  True, False,  True,  True,
        False, False,  True, False,  True, False, False, False,  True,  True,
         True,  True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True,  True, False, False, False, False,  True, False,  True,
        False, False, False, False, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True])

In [50]:
# 아래 tensor 를 sum 하면, 1 batch의 correct 수집
(pred.argmax(1) == y).type(torch.float)

tensor([1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [54]:
# batch size 64와 tensor size가 동일함을 알 수 있다.
(pred.argmax(1) == y).type(torch.float).size()

torch.Size([64])

위와 같은 방식으로 구한 `누적 correct`를 `len(dataloader.dataset)` 로 나눈 후 100을 곱하면 `Accuracy`를 구할 수 있다.